In [ ]:
import pandas as pd
import json

In [ ]:
price_history = pd.read_json('price_collection.json')
price_history.columns = ['Date', 'Ticker', 'Volume', 'Open', 'Low', 'High', 'Close']
price_history.sort_values('Date', ascending=True, inplace=True)

with open('lotsize.json', "r") as read_file:
    lotsize = json.load(read_file)
    read_file.close()

black_list = ('DSKY','LNZL','LNZLP', 'ARSA', 'ASSB', 'AVAN', 'BISVP', 'BRZL', 'BSPBP', 'CHGZ', 'DZRD', 'DZRDP', 'EELT', 'ELTZ', 'GEMA', 'IGST', 'IGSTP', 'JNOS', 'JNOSP', 'KBSB', 'KCHE', 'KCHEP', 'KGKC', 'KGKCP', 'KMEZ', 'KOGK', 'KRKN', 'KROTP', 'KRSB', 'KRSBP', 'KTSB', 'KTSBP', 'KUBE', 'KUZB', 'LPSB', 'LVHK', 'MAGE', 'MAGEP', 'MFGS', 'MFGSP', 'MGNZ', 'MGTS', 'MISB', 'MISB', 'MISBP', 'MRKK', 'MRSB', 'NFAZ', 'NKSH', 'NNSB', 'NNSBP', 'OMZZP', 'PRMB', 'RBCM', 'RDRB', 'RGSS', 'ROSB', 'RTGZ', 'RTSB', 'RTSBP', 'RUSI', 'RZSB', 'SAGO', 'SAGOP', 'SARE', 'SAREP', 'SIBG', 'SLEN', 'STSB', 'STSBP', 'SVET', 'TASB', 'TASBP', 'TGKDP', 'TNSE', 'TORS', 'TORSP', 'TUZA', 'UCSS', 'UKUZ', 'URKZ', 'USBN', 'UTAR', 'VGSB', 'VGSBP', 'VJGZ', 'VJGZP', 'VLHZ', 'VRSBP', 'WTCM', 'WTCMP', 'YKEN', 'YKENP', 'YRSB', 'YRSBP', 'ZILL')

In [ ]:
#FMBA ver1
idea_end = []
idea_active = []
idea_ready = []
#searching high volume over 10 ema
for selected_ticker in price_history.Ticker.unique():
    #create a short-list to count ema10 volume
    period_selection = price_history[(price_history['Ticker'] == selected_ticker) & (price_history['Date'] > '2023-01-01')]
    volume_check = period_selection['Volume'].to_frame()
    volume_check['Dayly_EMA10'] = volume_check['Volume'].ewm(span=10).mean()
    volume_check['increase'] = volume_check['Volume'] / volume_check['Dayly_EMA10'] 
    #reverse array to find last high-volume on condition x2 over ema10
    volume_check = volume_check[:9:-1]
    volume_check = volume_check[volume_check['increase'] > 2]
    #find a last volume at a green day
    first_day = 0
    i = 0
    if not volume_check.empty:        
        while (first_day == 0) and (i <= len(volume_check)-1):
            indx = volume_check.index[i]
            if period_selection.at[indx, 'Close'] > period_selection.at[indx, 'Open'] * 1.1:
                first_day = volume_check['increase'].index[i]
            i += 1
    if first_day > 0:        
        #counting emas
        check_ticker = period_selection['Close'].to_frame()
        dayly_ema = 'Dayly_EMA10'
        check_ticker[dayly_ema] = check_ticker['Close'].ewm(span=10).mean()   
        
        #counting ATR
        for i,tek in enumerate(check_ticker.index):
            if i == 0:
                check_ticker.at[tek,'TR'] = period_selection.at[tek,'High'] - period_selection.at[tek,'Low']
            else:
                pred = check_ticker.index[i-1]
                tr1 = period_selection.at[tek,'High'] - period_selection.at[tek,'Low']
                tr2 = abs(period_selection.at[tek,'High'] - period_selection.at[pred,'Close'])
                tr3 = abs(period_selection.at[pred,'Close'] - period_selection.at[tek,'Low'])
                check_ticker.at[tek,'TR'] = max(tr1, tr2, tr3)
        check_ticker['ATR'] = check_ticker['TR'].ewm(alpha=1/14).mean()
        
        
        check_ticker = check_ticker[check_ticker.index > first_day]
        for i in check_ticker.index:
            check_ticker.at[i, 'Low'] = period_selection.at[i, 'Low']
            check_ticker.at[i, 'High'] = period_selection.at[i, 'High']

        #check strategy
        buy_price = 0
        sell_price = 0
        stop_loss2 = 0
        for i,indx in enumerate(check_ticker.index):
            if buy_price == 0: 
                if (check_ticker.at[indx, 'Low'] <= check_ticker.at[indx, dayly_ema]):                    
                    lot_trade = period_selection.at[indx, 'Volume'] / lotsize.get(selected_ticker)
                    if lot_trade > 10000:
                        buy_price = check_ticker.at[indx, dayly_ema]                                                
            else:
                if sell_price == 0:                
                    take_profit = check_ticker.at[indx, dayly_ema] + 1.5 * check_ticker.at[indx, 'ATR']                    
                    stop_loss = buy_price - check_ticker.at[indx, 'ATR']                                        
                    if check_ticker.at[indx, 'High'] >= take_profit:
                        sell_price = take_profit
                        sell_date = period_selection.at[indx, 'Date']
                    elif check_ticker.at[indx, 'Low'] <= stop_loss:
                        sell_price = stop_loss
                        sell_date = period_selection.at[indx, 'Date']
        #indx = check_ticker.index[-1]        
        if sell_price > 0:
            idea_res = round((sell_price/buy_price-1)*100, 2)
            idea_end.append([selected_ticker, period_selection.at[first_day, 'Date'], sell_date, idea_res])
        elif buy_price > 0:
            idea_res = round((check_ticker.at[indx, 'Close']/buy_price-1)*100, 2)
            idea_active.append([period_selection.at[first_day, 'Date'], selected_ticker, idea_res])
        else:            
            lot_trade = period_selection.at[indx, 'Volume'] / lotsize.get(selected_ticker)
            if lot_trade > 10000:
                idea_ready.append(selected_ticker)
        
#print(idea_end)
print(idea_active)
print(idea_ready)

In [ ]:
mas = [0.049, 0.099, 0.124, 0.141, 0.165, 0.199, 0.249, 0.332, 0.499, 0.999]
for posi in mas:
    res = 1000000
    depo = res
    depo_fact = res
    active_posi = {}
    period = price_history[price_history['Date'] > '2022-11-01']
    for d in period.Date.unique():
        for share in idea_end:        
            if share[2] == d:
                tick_posi = active_posi.pop(share[0],0)            
                depo += tick_posi * share[3]/100
                depo_fact += tick_posi * (1+share[3]/100)
            elif share[1] == d and share[0] not in black_list:
                tick_posi = posi * depo
                if tick_posi <= depo_fact:
                    depo_fact -= tick_posi
                    active_posi[share[0]] = tick_posi
        if posi == 0.49:
            print(active_posi)
    print(posi, round(depo_fact,2), round(depo_fact/res,2))    